In [1]:
import requests
import pandas as pd

## Example of extracting features with the API

In [2]:
df = pd.read_csv("../data/outcomes_sampled_population_CONTUS_16_640_POP_100000_0.csv", index_col=0)
points = df[["lon", "lat"]].values
y_all = df["population"].values

### NAIP single point

In [3]:
%%time
lon, lat = points[0]

r = requests.post('http://msrcalebubuntu.eastus.cloudapp.azure.com:8080/featurizeNAIPSingle', json={
    'longitude': lon,
    'latitude': lat
})
status_code = r.status_code
response = r.json()
print(f"Response status code: {status_code}")
print(len(response["features"]))

Response status code: 200
1024
CPU times: user 6.89 ms, sys: 648 µs, total: 7.54 ms
Wall time: 1.3 s


### NAIP multiple points

In [4]:
%%time
lon0, lat0 = points[0]
lon1, lat1 = points[1]

r = requests.post('http://msrcalebubuntu.eastus.cloudapp.azure.com:8080/featurizeNAIPBatched', json={
    'longitudes': [lon0, lon1],
    'latitudes': [lat0, lat1]
})
status_code = r.status_code
response = r.json()
print(f"Response status code: {status_code}")
print(len(response["features"]))

Response status code: 200
2
CPU times: user 0 ns, sys: 4.48 ms, total: 4.48 ms
Wall time: 737 ms


### Sentinel 2 single point

In [5]:
%%time
lon, lat = points[0]

r = requests.post('http://msrcalebubuntu.eastus.cloudapp.azure.com:8080/featurizeSentinel2Single', json={
    'longitude': lon,
    'latitude': lat
})
status_code = r.status_code
response = r.json()
print(f"Response status code: {status_code}")
print(len(response["features"]))

Response status code: 200
1024
CPU times: user 4.78 ms, sys: 187 µs, total: 4.96 ms
Wall time: 2.38 s


## Examples of failure cases

In [6]:
## Missing parameter
r = requests.post('http://msrcalebubuntu.eastus.cloudapp.azure.com:8080/featurizeNAIPSingle', json={
    'latitude': 0
})
status_code = r.status_code
response = r.json()
print(f"Response status code: {status_code}")
print(response)

Response status code: 500
{'message': "'longitude' is a required parameter but wasn't sent"}


In [7]:
## Requested point is out of bounds
r = requests.post('http://msrcalebubuntu.eastus.cloudapp.azure.com:8080/featurizeNAIPSingle', json={
    'longitude': 0,
    'latitude': 0
})
status_code = r.status_code
response = r.json()
print(f"Response status code: {status_code}")
print(response)

Response status code: 500
{'message': 'No tile intersections'}


In [8]:
## Requested point is out of bounds
r = requests.post('http://msrcalebubuntu.eastus.cloudapp.azure.com:8080/featurizeNAIPBatched', json={
    'longitudes': [lon0, lon1],
    'latitudes': [lat0]
})
status_code = r.status_code
response = r.json()
print(f"Response status code: {status_code}")
print(response)

Response status code: 500
{'message': "The 'latitudes' and 'longitudes' inputs are not the same length"}


In [9]:
## Requested point is out of bounds
r = requests.post('http://msrcalebubuntu.eastus.cloudapp.azure.com:8080/featurizeNAIPBatched', json={
    'longitudes': list(range(1001)),
    'latitudes': list(range(1001))
})
status_code = r.status_code
response = r.json()
print(f"Response status code: {status_code}")
print(response)

Response status code: 500
{'message': 'The maximum number of points you can process at once is 1000'}
